In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import networkx as nx
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats as sps
import numpy as np
import igraph
from networkx.algorithms.community import modularity
import itertools
from collections import defaultdict, deque
from enum import Enum
import pandas as pd
# import random
# import walker

In [3]:
#!pip install 'python-graphblas[default]' 

In [4]:
from src.vcover_test import *
from src.tester_igraph import *
import src.domirank as domirank
import src.domirank_cg as domirank_cg
from src.reweights import compute_overlap_matrix, compute_overlap_overlap_matrix, compute_overlap_matrix_fast, compute_overlap_matrix_sq
from pathlib import Path

/home/niquepolice/DomiRankAdvanced/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [67]:
DOMI_TOL = 1e-8
VCOVER_RES_PATH = Path("results") / "vcover"
DESTRUCTION_RES_PATH = Path("results") / "destruction"
KMEANS_RES_PATH = Path("results") / "kmeans"

def domirank_vanilla(g, param=0.01):
    GAdj = g.get_adjacency_sparse()
    GAdj = GAdj.astype(float)
    A = GAdj
    lambN = domirank.find_eigenvalue_efficient(A)
    order = domirank_cg.calc_domirank(A, centrality=np.array(degree_centrality_ranking(g)), sigma=-param / lambN, tol=DOMI_TOL)
    return order


def domirank_on_reweigted(g, param=0.01, pow=1):
    GAdj = g.get_adjacency_sparse()
    GAdj = GAdj.astype(float)
    A = compute_overlap_matrix_fast(GAdj)
    lambN = domirank.find_eigenvalue_efficient(A)
    order = domirank_cg.calc_domirank(A, centrality=np.array(degree_centrality_ranking(g)), sigma=-param / lambN, tol=DOMI_TOL)
    return order

def domirank_on_reweigted_page(g, param=0.01):
    GAdj = g.get_adjacency_sparse()
    GAdj = GAdj.astype(float)
    A = compute_overlap_matrix_fast(GAdj)
    lambN = domirank.find_eigenvalue_efficient(A)
    order = domirank_cg.calc_domirank(A, centrality=np.array(pagerank_ranking(g)), sigma=-param / lambN, tol=DOMI_TOL)
    return order
    
def domirank_ov_nodeg(g, param=0.01):
    GAdj = g.get_adjacency_sparse()
    GAdj = GAdj.astype(float)
    A = compute_overlap_matrix_fast(GAdj, divide_by_deg=False)
    lambN = domirank.find_eigenvalue_efficient(A)
    order = domirank_cg.calc_domirank(A, centrality=np.array(degree_centrality_ranking(g)), sigma=-param / lambN, tol=DOMI_TOL)
    return order
    
def domirank_ov_nodeg_sq(g, param=0.01):
    GAdj = g.get_adjacency_sparse()
    GAdj = GAdj.astype(float)
    A = compute_overlap_matrix_sq(GAdj, divide_by_deg=False, use_adjacency_mask=False)
    lambN = -1  # A^2 + 2A = (A + I)^2 - I
    order = domirank_cg.calc_domirank(A, centrality=np.array(degree_centrality_ranking(g)), sigma=-param / lambN, tol=DOMI_TOL)
    return order

betweennesses = dict()
def betweenness_with_save(g):
    # if g.name in betweennesses:
        # betweenness = betweennesses[g.name]
    # else:
    betweenness = betweennesses[g.name] = betweenness_centrality_ranking(g)
    return betweenness

def domirank_on_reweigted_betweenness(g, param=0.01):
    GAdj = g.get_adjacency_sparse()
    GAdj = GAdj.astype(float)
    A = compute_overlap_matrix_fast(GAdj)
    lambN = domirank.find_eigenvalue_efficient(A)
    
    if g.name in betweennesses:
        betweenness = betweennesses[g.name]
    else:
        betweenness = betweennesses[g.name] = betweenness_centrality_ranking(g)
    betweenness = np.array(betweenness)
    
    
    order = domirank_cg.calc_domirank(A, centrality=betweenness, sigma=-param / lambN, tol=DOMI_TOL)
    return order

def nodes_num_skip(graph, func_name, max_nodes):
        n = graph.vcount()
        if n > max_nodes:
            print(f"⊘ Skipping {func_name} for {graph.name}: nodes num {n} > threshold {max_nodes}")
            return True
            
from src.tester_igraph import degree_centrality_ranking
def maxdeg_skip(graph, func_name, max_max_deg):
    max_deg = max(degree_centrality_ranking(graph))
    if max_deg > max_max_deg:
        print(f"⊘ Skipping {func_name} for {graph.name}: max degree {max_deg} > threshold {max_max_deg}")
        return True 

betweenness_skip = lambda graph, func_name: nodes_num_skip(graph, func_name, max_nodes=30000)
square_skip = lambda graph, func_name: maxdeg_skip(graph, func_name, max_max_deg=100)
            

    

In [68]:
# def voterank(g, k=None):
#     A = g.get_edgelist()
#     g_nx = nx.Graph(A)

#     top_k = nx.voterank(g_nx, k)
#     print(top_k)
#     print(np.asarray(top_k))


#     n = g.vcount()
#     print(np.arange(n, n - len(top_k), -1))
#     ranking = np.zeros(n)
#     ranking[np.asarray(top_k)] = np.arange(n, n - len(top_k), -1)

#     # use random order for the rest of nodes
    
    
#     return ranking 

In [69]:
classic_centralities = [
        ("Degree Centrality", degree_centrality_ranking),
        ("Betweenness Centrality", betweenness_centrality_ranking, betweenness_skip),
        # ("Closeness Centrality", closeness_centrality_ranking, 500),
        # ("Eigenvector Centrality", eigenvector_centrality_ranking),
        ("PageRank", pagerank_ranking),
        # ("VoteRank", voterank),

    ]

num_sigmas = 5
classic_domirank = [(f"DomiRank, sigma={sigma}", lambda g, param=sigma: domirank_vanilla(g, param=param)) for sigma in np.linspace(0.01, 0.99, num_sigmas)]    
domirank_on_overlapp = [(f"DomiRank Overlapping, sigma={sigma}", lambda g, param=sigma: domirank_on_reweigted(g, param=param)) for sigma in np.linspace(0.01, 0.99, num_sigmas)]    
domirank_on_overlapp_page = [(f"DomiRank Over+Pagerank, sigma={sigma}", lambda g, param=sigma: domirank_on_reweigted_page(g, param=param)) for sigma in np.linspace(0.01, 0.99, num_sigmas)]    
domirank_smth = [(f"DomiRankOverNodegSq, sigma={sigma}", lambda g, param=sigma: domirank_ov_nodeg_sq(g, param=param), square_skip) for sigma in np.linspace(0.01, 0.99, num_sigmas)]    
domirank_on_overlapp_bet = [(f"DomiRank Over+Betweenness, sigma={sigma}", lambda g, param=sigma: domirank_on_reweigted_betweenness(g, param=param), betweenness_skip) for sigma in np.linspace(0.01, 0.99, num_sigmas)] 

domis = [classic_domirank, domirank_on_overlapp, domirank_on_overlapp_page, domirank_smth, domirank_on_overlapp_bet]

In [70]:
def get_pd_dataframe_with_result(default_functions: list, choose_best_functions: list[list], graphs, graph_names, analizer):
    funcs_results = analizer(default_functions, graphs, graph_names, False)
    metrics, times = {}, {}
    for graph_name in graph_names:
        metrics[graph_name], times[graph_name] = {}, {}
        for funcs_result in funcs_results[graph_name]:
            func_name, metric = funcs_result["func_name"], funcs_result["metric"] 
            metrics[graph_name][func_name] = f"{round(metric, 1)}"
            times[graph_name][func_name] = funcs_result["runtime"] 
    for i in range(len(choose_best_functions)):
        temp_funcs_results = analizer(choose_best_functions[i], graphs, graph_names, False)
        temp_metrics = {}
        for graph_name in graph_names:
            temp_metrics[graph_name] = {}
            if graph_name not in times:
                times[graph_name] = {}
            for funcs_result in temp_funcs_results[graph_name]:
                func_name, metric = funcs_result["func_name"], funcs_result["metric"] 
                func_name_short = func_name.split(",")[0]
                temp_metrics[graph_name][func_name] = metric
                if func_name_short not in times[graph_name]:
                    times[graph_name][func_name_short] = 0
                times[graph_name][func_name_short] += funcs_result["runtime"]
        for graph_name in graph_names:
            if not temp_metrics[graph_name]:
                continue
            best_method = pd.DataFrame(temp_metrics)[graph_name].argmin()
            method_name_full = pd.DataFrame(temp_metrics).iloc[best_method].name
            method_name, param_str = method_name_full.split(", ")
            param_name, param_val = param_str.split("=") 
            
            metric_val = round(temp_metrics[graph_name][method_name_full], 1)
            text = f"({round(float(param_val), 2)}) {metric_val}"
            metrics[graph_name][method_name] = text 
    return pd.DataFrame(metrics), pd.DataFrame(times)

# def save_results(path, dataset_name):
#     df.to_csv(VCOVER_RES_PATH / "small.csv"), times_df.to_csv(VCOVER_RES_PATH / "small_times.csv")
    

In [71]:
graphs, graph_names = load_graphs_from_directory('data/small/', ['gml'])
df, times_df = get_pd_dataframe_with_result(classic_centralities, domis, graphs, graph_names, batch_vertex_cover_analysis)
if domirank_on_overlapp_bet in domis and betweenness_with_save in classic_centralities:
    times_df.loc["DomiRank Over+Betweenness"] += times_df.loc["Betweenness Centrality"]
df.to_csv(VCOVER_RES_PATH / "small.csv"), times_df.to_csv(VCOVER_RES_PATH / "small_times.csv")
pd.read_csv(VCOVER_RES_PATH / "small.csv", index_col=0).transpose()

Loaded graph: barabasi_albert_10000_3.gml (10000 nodes, 29991 edges)
Loaded graph: chicago_road.gml (12979 nodes, 20627 edges)


100%|██████████| 5/5 [00:00<00:00, 2127.79it/s]


⊘ Skipping DomiRankOverNodegSq, sigma=0.01 for barabasi_albert_10000_3.gml: max degree 222 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.255 for barabasi_albert_10000_3.gml: max degree 222 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.5 for barabasi_albert_10000_3.gml: max degree 222 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.745 for barabasi_albert_10000_3.gml: max degree 222 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.99 for barabasi_albert_10000_3.gml: max degree 222 > threshold 100


100%|██████████| 5/5 [00:01<00:00,  2.83it/s]


100%|██████████| 5/5 [00:11<00:00,  2.22s/it]


,Degree Centrality,Betweenness Centrality,PageRank,DomiRank,DomiRank Overlapping,DomiRank Over+Pagerank,DomiRank Over+Betweenness,DomiRankOverNodegSq
barabasi_albert_10000_3.gml,371.6,369.2,357.9,(0.01) 363.5,(0.01) 363.8,(0.01) 358.1,(0.26) 359.7,NaN
chicago_road.gml,2529.4,3495.7,2079.4,(0.74) 1833.4,(0.99) 1869.3,(0.74) 1786.8,(0.99) 2790.3,(0.5) 1575.1


In [72]:
graphs, graph_names = load_graphs_from_directory('data/medium/', ['gml'])

Loaded graph: marvel_universe.gml (19182 nodes, 95445 edges)
Loaded graph: python_dependency.gml (58302 nodes, 107819 edges)


In [73]:
df, times_df = get_pd_dataframe_with_result(classic_centralities, domis, graphs, graph_names, batch_vertex_cover_analysis)
if domirank_on_overlapp_bet in domis and betweenness_with_save in classic_centralities:
    times_df.loc["DomiRank Over+Betweenness"] += times_df.loc["Betweenness Centrality"]
df.to_csv(VCOVER_RES_PATH / "med.csv"), times_df.to_csv(VCOVER_RES_PATH / "med_times.csv")
pd.read_csv(VCOVER_RES_PATH / "med.csv", index_col=0).transpose()

100%|██████████| 3/3 [00:00<00:00, 10.51it/s]


⊘ Skipping Betweenness Centrality for python_dependency.gml: nodes num 58302 > threshold 30000


100%|██████████| 5/5 [00:00<00:00, 1796.74it/s]


⊘ Skipping DomiRankOverNodegSq, sigma=0.01 for marvel_universe.gml: max degree 1625 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.255 for marvel_universe.gml: max degree 1625 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.5 for marvel_universe.gml: max degree 1625 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.745 for marvel_universe.gml: max degree 1625 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.99 for marvel_universe.gml: max degree 1625 > threshold 100


100%|██████████| 5/5 [00:00<00:00, 834.55it/s]


⊘ Skipping DomiRankOverNodegSq, sigma=0.01 for python_dependency.gml: max degree 35944 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.255 for python_dependency.gml: max degree 35944 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.5 for python_dependency.gml: max degree 35944 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.745 for python_dependency.gml: max degree 35944 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.99 for python_dependency.gml: max degree 35944 > threshold 100


100%|██████████| 5/5 [00:00<00:00, 16487.04it/s]

⊘ Skipping DomiRank Over+Betweenness, sigma=0.01 for python_dependency.gml: nodes num 58302 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.255 for python_dependency.gml: nodes num 58302 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.5 for python_dependency.gml: nodes num 58302 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.745 for python_dependency.gml: nodes num 58302 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.99 for python_dependency.gml: nodes num 58302 > threshold 30000


,Degree Centrality,Betweenness Centrality,PageRank,DomiRank,DomiRank Overlapping,DomiRank Over+Pagerank,DomiRank Over+Betweenness
marvel_universe.gml,1200.2,567.6,793.0,(0.74) 1091.2,(0.74) 980.1,(0.01) 791.1,(0.01) 559.5
python_dependency.gml,351.4,NaN,219.1,(0.01) 354.5,(0.74) 307.2,(0.01) 219.1,NaN


In [74]:
graphs, graph_names = load_graphs_from_directory('data/large/', ['gml'])
df, times_df = get_pd_dataframe_with_result(classic_centralities, domis, graphs, graph_names, batch_vertex_cover_analysis)
if domirank_on_overlapp_bet in domis and betweenness_with_save in classic_centralities:
    times_df.loc["DomiRank Over+Betweenness"] += times_df.loc["Betweenness Centrality"]
df.to_csv(VCOVER_RES_PATH / "large.csv"), times_df.to_csv(VCOVER_RES_PATH / "large_times.csv")
pd.read_csv(VCOVER_RES_PATH / "large.csv", index_col=0).transpose()

Loaded graph: foursquare_friendship.gml (105091 nodes, 357921 edges)
Loaded graph: AstroPh.gml (17903 nodes, 196972 edges)


 33%|███▎      | 1/3 [00:00<00:00,  3.08it/s]

⊘ Skipping Betweenness Centrality for foursquare_friendship.gml: nodes num 105091 > threshold 30000


100%|██████████| 5/5 [00:00<00:00, 171.54it/s]


⊘ Skipping DomiRankOverNodegSq, sigma=0.01 for foursquare_friendship.gml: max degree 692 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.255 for foursquare_friendship.gml: max degree 692 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.5 for foursquare_friendship.gml: max degree 692 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.745 for foursquare_friendship.gml: max degree 692 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.99 for foursquare_friendship.gml: max degree 692 > threshold 100


...

⊘ Skipping DomiRankOverNodegSq, sigma=0.01 for AstroPh.gml: max degree 504 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.255 for AstroPh.gml: max degree 504 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.5 for AstroPh.gml: max degree 504 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.745 for AstroPh.gml: max degree 504 > threshold 100
⊘ Skipping DomiRankOverNodegSq, sigma=0.99 for AstroPh.gml: max degree 504 > threshold 100


100%|██████████| 5/5 [00:00<00:00, 46603.38it/s]


⊘ Skipping DomiRank Over+Betweenness, sigma=0.01 for foursquare_friendship.gml: nodes num 105091 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.255 for foursquare_friendship.gml: nodes num 105091 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.5 for foursquare_friendship.gml: nodes num 105091 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.745 for foursquare_friendship.gml: nodes num 105091 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.99 for foursquare_friendship.gml: nodes num 105091 > threshold 30000


100%|██████████| 5/5 [00:52<00:00, 10.41s/it]


,Degree Centrality,PageRank,DomiRank,DomiRank Overlapping,DomiRank Over+Pagerank,Betweenness Centrality,DomiRank Over+Betweenness
foursquare_friendship.gml,14917.3,9881.8,(0.74) 12455.6,(0.99) 11740.4,(0.99) 8858.2,NaN,NaN
AstroPh.gml,1976.7,1079.6,(0.74) 1404.0,(0.74) 1271.5,(0.74) 763.7,671.8,(0.5) 601.2


In [75]:
graphs, graph_names = load_graphs_from_directory('data/add/', ['gml'])
df, times_df = get_pd_dataframe_with_result(classic_centralities, domis, graphs, graph_names, batch_vertex_cover_analysis)
if domirank_on_overlapp_bet in domis and betweenness_with_save in classic_centralities:
    times_df.loc["DomiRank Over+Betweenness"] += times_df.loc["Betweenness Centrality"]
df.to_csv(VCOVER_RES_PATH / "add.csv"), times_df.to_csv(VCOVER_RES_PATH / "add_times.csv")
pd.read_csv(VCOVER_RES_PATH / "add.csv", index_col=0).transpose()

Loaded graph: rgg_10000_0_02.gml (10000 nodes, 61717 edges)
Loaded graph: roadnet-TX.gml (1351137 nodes, 1879201 edges)
Loaded graph: roadnet-CA.gml (1957027 nodes, 2760388 edges)
Loaded graph: rgg_1000_0_05.gml (999 nodes, 3644 edges)
Loaded graph: rgg_10000_0_03.gml (10000 nodes, 136356 edges)
Loaded graph: ahmedabad.gml (2870 nodes, 4375 edges)
Loaded graph: rgg_1000_0_15.gml (1000 nodes, 31151 edges)
Loaded graph: us_power_grid.gml (4941 nodes, 6594 edges)
Loaded graph: caida-20071112.gml (26389 nodes, 52861 edges)
Loaded graph: euroroad.gml (1039 nodes, 1305 edges)
Loaded graph: rgg_10000_0_01.gml (196 nodes, 313 edges)
Loaded graph: rgg_1000_0_10.gml (1000 nodes, 14621 edges)


/home/niquepolice/DomiRankAdvanced/src/tester_igraph.py:427: RuntimeWarning: One or more unknown entities will be returned verbatim (&NewLine;). Location: src/io/gml.c:149
  g = ig.Graph.Read_GML(file_path)


Loaded graph: roadnet_PA.gml (1087562 nodes, 1541514 edges)


 33%|███▎      | 1/3 [00:03<00:06,  3.05s/it]

⊘ Skipping Betweenness Centrality for roadnet-TX.gml: nodes num 1351137 > threshold 30000


...

⊘ Skipping DomiRank Over+Betweenness, sigma=0.01 for roadnet-CA.gml: nodes num 1957027 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.255 for roadnet-CA.gml: nodes num 1957027 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.5 for roadnet-CA.gml: nodes num 1957027 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.745 for roadnet-CA.gml: nodes num 1957027 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.99 for roadnet-CA.gml: nodes num 1957027 > threshold 30000


100%|██████████| 5/5 [00:00<00:00, 10.14it/s]


100%|██████████| 5/5 [00:00<00:00, 31254.13it/s]

⊘ Skipping DomiRank Over+Betweenness, sigma=0.01 for roadnet_PA.gml: nodes num 1087562 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.255 for roadnet_PA.gml: nodes num 1087562 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.5 for roadnet_PA.gml: nodes num 1087562 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.745 for roadnet_PA.gml: nodes num 1087562 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.99 for roadnet_PA.gml: nodes num 1087562 > threshold 30000


,Degree Centrality,Betweenness Centrality,PageRank,DomiRank,DomiRank Overlapping,DomiRank Over+Pagerank,DomiRankOverNodegSq,DomiRank Over+Betweenness
rgg_10000_0_02.gml,1866.8,1708.7,1041.5,(0.99) 832.6,(0.74) 758.4,(0.99) 719.4,(0.74) 591.9,(0.99) 1096.3
roadnet-TX.gml,363347.7,NaN,261334.2,(0.99) 219177.6,(0.99) 235589.3,(0.99) 231542.0,(0.5) 198124.9,NaN
roadnet-CA.gml,523387.5,NaN,370578.0,(0.99) 314775.8,(0.99) 333525.9,(0.99) 331148.2,(0.5) 284044.9,NaN
rgg_1000_0_05.gml,214.5,202.8,124.0,(0.74) 115.6,(0.74) 102.0,(0.74) 96.2,(0.5) 80.0,(0.99) 131.6
rgg_10000_0_03.gml,1437.7,1459.5,724.8,(0.74) 448.9,(0.74) 435.2,(0.74) 430.7,(0.99) 335.9,(0.99) 814.2
ahmedabad.gml,794.7,879.3,584.9,(0.74) 458.1,(0.74) 463.1,(0.74) 478.1,(0.5) 418.7,(0.99) 695.3
rgg_1000_0_15.gml,96.8,113.0,55.2,(0.74) 26.7,(0.74) 27.1,(0.74) 33.3,(0.99) 19.3,(0.99) 29.3
us_power_grid.gml,823.3,980.7,650.7,(0.99) 644.9,(0.99) 652.6,(0.99) 606.7,(0.26) 552.1,(0.99) 837.8
caida-20071112.gml,384.6,246.0,284.8,(0.01) 348.1,(0.01) 347.0,(0.01) 284.8,NaN,(0.01) 244.9
euroroad.gml,236.1,279.8,169.4,(0.74) 164.9,(0.74) 170.0,(0.74) 153.6,(0.26) 138.6,(0.99) 219.9


In [76]:
graphs, graph_names = load_graphs_from_directory('data/usa/', ['gml'])
df, times_df = get_pd_dataframe_with_result(classic_centralities, domis, graphs, graph_names, batch_vertex_cover_analysis)
df.to_csv(VCOVER_RES_PATH / "usa.csv"), times_df.to_csv(VCOVER_RES_PATH / "usa_times.csv")

Loaded graph: merged.gml (23947347 nodes, 28854312 edges)


 33%|███▎      | 1/3 [00:47<01:35, 47.61s/it]

⊘ Skipping Betweenness Centrality for merged.gml: nodes num 23947347 > threshold 30000


 80%|████████  | 4/5 [19:31<06:16, 376.64s/it]

100%|██████████| 5/5 [00:00<00:00, 45100.04it/s]

⊘ Skipping DomiRank Over+Betweenness, sigma=0.01 for merged.gml: nodes num 23947347 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.255 for merged.gml: nodes num 23947347 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.5 for merged.gml: nodes num 23947347 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.745 for merged.gml: nodes num 23947347 > threshold 30000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.99 for merged.gml: nodes num 23947347 > threshold 30000


(None, None)

In [77]:
names = ["small", "med", "large", "add", "usa"]
all_df = pd.concat([pd.read_csv(VCOVER_RES_PATH / f"{dir}.csv", index_col=0) for dir in names], axis=1).transpose()
all_df.to_csv(VCOVER_RES_PATH / "all.csv")
all_df

,Degree Centrality,Betweenness Centrality,PageRank,DomiRank,DomiRank Overlapping,DomiRank Over+Pagerank,DomiRank Over+Betweenness,DomiRankOverNodegSq
barabasi_albert_10000_3.gml,371.6,369.2,357.9,(0.01) 363.5,(0.01) 363.8,(0.01) 358.1,(0.26) 359.7,NaN
chicago_road.gml,2529.4,3495.7,2079.4,(0.74) 1833.4,(0.99) 1869.3,(0.74) 1786.8,(0.99) 2790.3,(0.5) 1575.1
marvel_universe.gml,1200.2,567.6,793.0,(0.74) 1091.2,(0.74) 980.1,(0.01) 791.1,(0.01) 559.5,NaN
python_dependency.gml,351.4,NaN,219.1,(0.01) 354.5,(0.74) 307.2,(0.01) 219.1,NaN,NaN
foursquare_friendship.gml,14917.3,NaN,9881.8,(0.74) 12455.6,(0.99) 11740.4,(0.99) 8858.2,NaN,NaN
AstroPh.gml,1976.7,671.8,1079.6,(0.74) 1404.0,(0.74) 1271.5,(0.74) 763.7,(0.5) 601.2,NaN
rgg_10000_0_02.gml,1866.8,1708.7,1041.5,(0.99) 832.6,(0.74) 758.4,(0.99) 719.4,(0.99) 1096.3,(0.74) 591.9
roadnet-TX.gml,363347.7,NaN,261334.2,(0.99) 219177.6,(0.99) 235589.3,(0.99) 231542.0,NaN,(0.5) 198124.9
roadnet-CA.gml,523387.5,NaN,370578.0,(0.99) 314775.8,(0.99) 333525.9,(0.99) 331148.2,NaN,(0.5) 284044.9
rgg_1000_0_05.gml,214.5,202.8,124.0,(0.74) 115.6,(0.74) 102.0,(0.74) 96.2,(0.99) 131.6,(0.5) 80.0


In [78]:
all_df_times = pd.concat([pd.read_csv(VCOVER_RES_PATH / f"{dir}_times.csv", index_col=0) for dir in names], axis=1).transpose()
all_df_times.to_csv(VCOVER_RES_PATH / "all_times.csv")
all_df_times

,Degree Centrality,Betweenness Centrality,PageRank,DomiRank,DomiRank Overlapping,DomiRank Over+Pagerank,DomiRank Over+Betweenness,DomiRankOverNodegSq
barabasi_albert_10000_3.gml,0.000097,7.662016,0.006284,0.294569,0.377530,0.865130,8.048536,NaN
chicago_road.gml,0.000125,9.688996,0.009475,1.186447,1.273934,1.579076,10.784056,1.468047
marvel_universe.gml,0.000196,37.148883,0.010572,1.235187,1.356461,1.478132,37.988973,NaN
python_dependency.gml,0.000551,NaN,0.020487,1.214229,1.262867,1.504086,NaN,NaN
foursquare_friendship.gml,0.000957,NaN,0.045238,4.500012,3.695516,3.113328,NaN,NaN
AstroPh.gml,0.000513,48.610737,0.018866,1.994307,2.290958,2.302260,51.371333,NaN
rgg_10000_0_02.gml,0.000124,12.601051,0.006962,0.719078,1.084318,1.435358,13.170897,3.589610
roadnet-TX.gml,0.011811,NaN,0.337824,117.950392,39.504694,32.506959,NaN,158.238917
roadnet-CA.gml,0.016796,NaN,0.505144,184.541163,66.392782,53.258348,NaN,223.992997
rgg_1000_0_05.gml,0.000045,0.076224,0.003198,0.068404,0.101664,0.655551,0.163636,0.337832


## Batch destruction

In [48]:
graphs, graph_names = load_graphs_from_directory('data/small/', ['gml'])

Loaded graph: barabasi_albert_10000_3.gml (10000 nodes, 29991 edges)
Loaded graph: chicago_road.gml (12979 nodes, 20627 edges)


In [49]:
df, times_df = get_pd_dataframe_with_result(classic_centralities, domis, graphs, graph_names, batch_destruction_analysis)
if domirank_on_overlapp_bet in domis and betweenness_with_save in classic_centralities:
    times_df.loc["DomiRank Over+Betweenness"] += times_df.loc["Betweenness Centrality"]
df.to_csv(DESTRUCTION_RES_PATH / "small.csv"), times_df.to_csv(DESTRUCTION_RES_PATH / "small_times.csv")
pd.read_csv(DESTRUCTION_RES_PATH / "small.csv", index_col=0).transpose()

100%|██████████| 3/3 [00:00<00:00, 28.18it/s]


⊘ Skipping Betweenness Centrality for barabasi_albert_10000_3.gml: nodes num 10000 > treshold 2000


100%|██████████| 3/3 [00:00<00:00, 30.49it/s]


⊘ Skipping Betweenness Centrality for chicago_road.gml: nodes num 12979 > treshold 2000


100%|██████████| 2/2 [00:00<00:00, 3766.78it/s]


...

100%|██████████| 2/2 [00:00<00:00, 7416.98it/s]


⊘ Skipping DomiRank Over+Betweenness, sigma=0.01 for barabasi_albert_10000_3.gml: nodes num 10000 > treshold 2000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.99 for barabasi_albert_10000_3.gml: nodes num 10000 > treshold 2000


100%|██████████| 2/2 [00:00<00:00, 7619.08it/s]

⊘ Skipping DomiRank Over+Betweenness, sigma=0.01 for chicago_road.gml: nodes num 12979 > treshold 2000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.99 for chicago_road.gml: nodes num 12979 > treshold 2000


,Degree Centrality,PageRank,DomiRank,DomiRank Overlapping,DomiRank Over+Pagerank,DomiRankOverNodegSq
barabasi_albert_10000_3.gml,1890.6,1853.2,(0.99) 1885.7,(0.01) 1894.2,(0.01) 1853.2,NaN
chicago_road.gml,2556.2,2557.9,(0.99) 2598.3,(0.99) 2326.3,(0.99) 2557.9,(0.01) 2595.7


In [50]:
graphs, graph_names = load_graphs_from_directory('data/medium/', ['gml'])

Loaded graph: marvel_universe.gml (19182 nodes, 95445 edges)
Loaded graph: python_dependency.gml (58302 nodes, 107819 edges)


In [51]:
df, times_df = get_pd_dataframe_with_result(classic_centralities, domis, graphs, graph_names, batch_destruction_analysis)
if domirank_on_overlapp_bet in domis and betweenness_with_save in classic_centralities:
    times_df.loc["DomiRank Over+Betweenness"] += times_df.loc["Betweenness Centrality"]
df.to_csv(DESTRUCTION_RES_PATH / "med.csv"), times_df.to_csv(DESTRUCTION_RES_PATH / "med_times.csv")
pd.read_csv(DESTRUCTION_RES_PATH / "med.csv", index_col=0).transpose()

  0%|          | 0/3 [00:00<?, ?it/s]

⊘ Skipping Betweenness Centrality for marvel_universe.gml: nodes num 19182 > treshold 2000


 33%|███▎      | 1/3 [00:00<00:00,  7.94it/s]

⊘ Skipping Betweenness Centrality for python_dependency.gml: nodes num 58302 > treshold 2000


100%|██████████| 2/2 [00:00<00:00, 1754.94it/s]


...

100%|██████████| 2/2 [00:00<00:00, 35544.95it/s]


⊘ Skipping DomiRank Over+Betweenness, sigma=0.01 for marvel_universe.gml: nodes num 19182 > treshold 2000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.99 for marvel_universe.gml: nodes num 19182 > treshold 2000


100%|██████████| 2/2 [00:00<00:00, 38304.15it/s]

⊘ Skipping DomiRank Over+Betweenness, sigma=0.01 for python_dependency.gml: nodes num 58302 > treshold 2000
⊘ Skipping DomiRank Over+Betweenness, sigma=0.99 for python_dependency.gml: nodes num 58302 > treshold 2000


,Degree Centrality,PageRank,DomiRank,DomiRank Overlapping,DomiRank Over+Pagerank
marvel_universe.gml,1766.6,1556.2,(0.01) 1766.6,(0.01) 1773.4,(0.01) 1556.2
python_dependency.gml,310.6,267.0,(0.01) 310.7,(0.01) 311.4,(0.01) 267.0


In [52]:
names = ["small", "med"]#, "large", "add", "usa"]
all_df = pd.concat([pd.read_csv(DESTRUCTION_RES_PATH / f"{dir}.csv", index_col=0) for dir in names], axis=1).transpose()
all_df.to_csv(DESTRUCTION_RES_PATH / "all.csv")
all_df

,Degree Centrality,PageRank,DomiRank,DomiRank Overlapping,DomiRank Over+Pagerank,DomiRankOverNodegSq
barabasi_albert_10000_3.gml,1890.6,1853.2,(0.99) 1885.7,(0.01) 1894.2,(0.01) 1853.2,NaN
chicago_road.gml,2556.2,2557.9,(0.99) 2598.3,(0.99) 2326.3,(0.99) 2557.9,(0.01) 2595.7
marvel_universe.gml,1766.6,1556.2,(0.01) 1766.6,(0.01) 1773.4,(0.01) 1556.2,NaN
python_dependency.gml,310.6,267.0,(0.01) 310.7,(0.01) 311.4,(0.01) 267.0,NaN


In [53]:
all_df_times = pd.concat([pd.read_csv(DESTRUCTION_RES_PATH / f"{dir}_times.csv", index_col=0) for dir in names], axis=1).transpose()
all_df_times.to_csv(DESTRUCTION_RES_PATH / "all_times.csv")
all_df_times

,Degree Centrality,PageRank,DomiRank,DomiRank Overlapping,DomiRank Over+Pagerank,DomiRankOverNodegSq
barabasi_albert_10000_3.gml,0.000118,0.009205,0.091409,0.102202,0.323926,NaN
chicago_road.gml,0.000160,0.005338,0.286042,0.295279,0.505604,0.059781
marvel_universe.gml,0.000183,0.010782,0.224340,0.327736,0.551208,NaN
python_dependency.gml,0.000539,0.020523,0.262980,0.263949,0.483346,NaN
